In [1]:
import os 
os.chdir("../")

In [2]:
os.getcwd()

'c:\\Users\\omrav\\OneDrive\\Desktop\\IITC AI\\Prep\\mlops projects\\Chest-Cancer-Classification-using-MLFlow-and-DVC'

In [3]:
import dagshub
dagshub.init(repo_owner='Prashanth0205', repo_name='Chest-Cancer-Classification-using-MLFlow-and-DVC', mlflow=True)

Accessing as Prashanth0205

Initialized MLflow to track repo "Prashanth0205/Chest-Cancer-Classification-using-MLFlow-and-DVC"

Repository Prashanth0205/Chest-Cancer-Classification-using-MLFlow-and-DVC initialized!

In [ ]:
# import mlflow
# with mlflow.start_run():
#   mlflow.log_param('parameter name', 'value')
#   mlflow.log_metric('metric name', 1)

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict 
    mlflow_uri: str
    params_device: str
    params_batch_size: int

In [5]:
from src.CNNClassifier.constants import * 
from src.CNNClassifier.utils.common import read_yaml, create_directories, save_json

In [6]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model=self.config.training.trained_model_path,
            training_data=Path("artifacts\data_ingestion\CT-Scan"),
            mlflow_uri="https://dagshub.com/Prashanth0205/Chest-Cancer-Classification-using-MLFlow-and-DVC.mlflow",
            all_params=self.params,
            params_device=self.params.DEVICE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import mlflow
from urllib.parse import urlparse

In [12]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config 

    def get_trained_model(self):
        self.device = self.config.params_device
        self.model = torch.load(self.config.path_of_model)
        self.model = self.model.to(self.device)

    def test_generator(self):
        test_data_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

        test_dataset = datasets.ImageFolder(root=Path(f'{self.config.training_data}/test'), transform=test_data_transform)
        self.test_loader = DataLoader(test_dataset, batch_size=self.config.params_batch_size, shuffle=False)

    def test(self):
        criterion = nn.CrossEntropyLoss() 
        total_loss = 0.0
        running_corrects = 0

        with torch.no_grad():
            for inputs, labels in self.test_loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)

                outputs = self.model(inputs)
                loss = criterion(outputs, labels)  
                _, preds = torch.max(outputs, 1)

                total_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels)

        self.loss = total_loss / len(self.test_loader.dataset)
        self.accuracy = running_corrects/ len(self.test_loader.dataset)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.loss, "accuracy": self.accuracy}

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"loss": self.loss, "accuracy": self.accuracy})

            if tracking_url_type_store != "file":
                mlflow.pytorch.log_model(self.model, "model", registered_model_name='VGG16_PytorchModel')
            else:
                mlflow.pytorch.log_model(self.model, "model")


In [13]:
try: 
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.get_trained_model()
    evaluation.test_generator()
    evaluation.test()
    evaluation.log_into_mlflow()
    
except Exception as e:
    raise e 

[2024-10-25 00:16:37,676: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-25 00:16:37,680: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-25 00:16:37,682: INFO: common: created directory at: artifacts]


2024/10/25 00:17:58 WARNING mlflow.utils.requirements_utils: Found torch version (2.0.0+cu117) contains a local version label (+cu117). MLflow logged a pip requirement for this package as 'torch==2.0.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2024/10/25 00:18:08 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.15.0+cu117) contains a local version label (+cu117). MLflow logged a pip requirement for this package as 'torchvision==0.15.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2024/10/25 00:18:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'VGG16_P